In [1]:
from reinforcetrader.data import RawDataLoader

In [ ]:
dataloader = RawDataLoader(episode_config_pth='configs/episodes.json')
dataloader.get_hist_prices()

[**********************88%*****************      ]  440 of 501 completed